In [1]:
from pathlib import Path

import yaml

from massivesearch.ext.pandas.aggregator import PandasAggregator
from massivesearch.ext.pandas.number import PandasNumberSearchEngine
from massivesearch.ext.pandas.text import PandasTextSearchEngine
from massivesearch.ext.pandas.types import PandasAggregatorConfig
from massivesearch.index.number import BasicNumberIndex
from massivesearch.index.text import BasicTextIndex
from massivesearch.model.azure_openai import AzureOpenAIClient
from massivesearch.spec import SpecBuilder
from massivesearch.worker import Worker


PydanticSchemaGenerationError: Unable to generate pydantic-core schema for <class 'massivesearch.ext.pandas.types.PandasAggregatorConfig'>. Set `arbitrary_types_allowed=True` in the model_config to ignore this error or implement `__get_pydantic_core_schema__` on your type to fully support it.

If you got this error by calling handler(<some type>) within `__get_pydantic_core_schema__` then you likely need to call `handler.generate_schema(<some type>)` since we do not call `__get_pydantic_core_schema__` on `<some type>` otherwise to avoid infinite recursion.

For further information visit https://errors.pydantic.dev/2.11/u/schema-for-unknown-type

In [2]:
book_builder = SpecBuilder()

book_builder.register_index("text_index", BasicTextIndex)
book_builder.register_index("number_index", BasicNumberIndex)
book_builder.register_search_engine("book_text_search", PandasTextSearchEngine)
book_builder.register_search_engine("book_price_search", PandasNumberSearchEngine)

If you have customed the index and search engine, you can also use the decorator:

```python

@book_builder.index("text_index")
class BookTextIndex(BasicTextIndex):
    """Book text index."""


@book_builder.index("number_index")
class BookPriceIndex(BasicNumberIndex):
    """Book price index."""


@book_builder.search_engine("book_text_search")
class BookTextSearchEngine(PandasTextSearchEngine):
    """Book text search engine."""


@book_builder.search_engine("book_price_search")
class BookPriceSearchEngine(PandasNumberSearchEngine):
    """Book price search engine."""

```

In [ ]:
with Path("book_spec.yaml").open() as file:
        spec_file = yaml.safe_load(file)

book_builder.include(spec_file)
worker = Worker(
    book_builder.spec,
    model_client=AzureOpenAIClient(temperature=0),
)
execute_results = worker.search(
    "I want to buy a book about prince or lord, and I only have 20 dollars.",
)
agg = PandasAggregator(config=PandasAggregatorConfig(file_path="books.csv"))
agg_result = agg.aggregate(execute_results)

NameError: name 'Path' is not defined

Check Query:

In [ ]:
for result in execute_results:
    print(result.query)

Check Result:

In [ ]:
agg_results